https://python.langchain.com/v0.1/docs/use_cases/graph/constructing/

In [ ]:
from langchain_community.graphs import Neo4jGraph

url = "neo4j://localhost:7687"
username = "neo4j"
password = "anatomy-exodus-pigment-colombo-octopus-1216"

graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

In [23]:
import requests

url = "https://www.lguplus.com/mobile/device/phone"

r = requests.get(url)
content = r.content.decode("utf-8")
content

'<!doctype html><html data-n-head-ssr lang="ko" data-n-head="%7B%22lang%22:%7B%22ssr%22:%22ko%22%7D%7D"><head ><title>휴대폰</title><meta data-n-head="ssr" name="viewport" content="width=device-width, initial-scale=0.48, maximum-scale=0.5, minimum-scale=0.3"><meta data-n-head="ssr" charset="utf-8"><meta data-n-head="ssr" http-equiv="X-UA-Compatible" content="IE=edge"><meta data-n-head="ssr" data-hid="description" name="description" content="휴대폰"><meta data-n-head="ssr" name="format-detection" content="telephone=no,address=no,email=no"><meta data-n-head="ssr" name="google-site-verification" content="FcOenpICu_iLLI1NF8NRwXPUsHOenpyScAfvcLS6_pM"><meta data-n-head="ssr" name="google-site-verification" content="-4XQrz2ukrgYIWKr31A0EDEpr4uIRVO2YvSYEjuMRXQ"><meta data-n-head="ssr" name="google-site-verification" content="QzpaVh18wRCxJmeKnLFdfJXYdNvXaSHWgqbqF_wKOQo"><meta data-n-head="ssr" name="naver-site-verification" content="756c5f0d06b8e657fba476b6ffa0de88efa8caa8"><meta data-n-head="ssr" na

In [29]:
from langchain_text_splitters import HTMLHeaderTextSplitter, HTMLSemanticPreservingSplitter
from bs4 import Tag

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
]

def code_handler(element: Tag) -> str:
    data_lang = element.get("data-lang")
    code_format = f"<code:{data_lang}>{element.get_text()}</code>"

    return code_format

splitter = HTMLSemanticPreservingSplitter(
    headers_to_split_on=headers_to_split_on,
    separators=["\n\n", "\n", ". ", "! ", "? "],
    max_chunk_size=50,
    preserve_images=True,
    preserve_videos=True,
    elements_to_preserve=["table", "ul", "ol", "code"],
    denylist_tags=["script", "style", "head"],
    custom_handlers={"code": code_handler},
)

documents = splitter.split_text(content)
documents

[Document(metadata={}, page_content='본문 내용으로 건너뛰기 ![image:]() 오늘 하루 그만보기 닫기 상단 정보영역 LG U+ 개인 기업 소상공인 모바일 모바일 기기 휴대폰 중고폰 자급제폰 자급제폰 등록 태블릿 스마트 워치 휴대폰 지원금 모바일 요금제 5G/LTE 온라인 단독 요금제 태블릿/스마트워치 자급제폰 등록 듀얼넘버 플러스 유심/eSIM 유심 eSIM 모바일 결합할인 다이렉트 플러스 결합 U⁺투게더 결합 참 쉬운 가족 결합 U⁺가족무한사랑 참 쉬운 케이블 가족 결합 모바일 부가서비스 전체 데이터 영상/음악 통화/문자메시지 할인/편의 금융/보안 안심/보험 해외로밍 해외로밍 홈 로밍 요금제 신청 로밍 이용 조회 로밍 부가서비스 공지사항 인터넷/IPTV 인터넷+IPTV 가입 홈 상품 가입 가입 사은품 홈 상품 기획전 인터넷 요금제 부가 서비스 IPTV 요금제 채널 안내 주요 서비스 부가 서비스 공지사항 리모컨 구매 스마트홈 요금제 주요 기기 홈 결합할인 인터넷+IPTV+스마트홈 인터넷 가족끼리 모여 할인 U⁺투게더 결합 참 쉬운 가족결합 전화서비스 인터넷 전화 요금제 인터넷 전화 부가서비스 인터넷 전화 상품 국제전화 002 소상공인상품 소상공인상품 홈 맞춤 결합 요금제 매장 AX 솔루션 혜택/이벤트 소상공인 Insight 마이페이지 요금/납부 가입/사용 현황 가입 정보 조회/변경 요금제 조회/변경 부가서비스 조회/변경/해지 사용내역 조회 해외로밍 조회 휴대폰 결제 일시정지/해제 서비스 해지 회원 정보 데이터 선물/충전 참 쉬운 가족 데이터 데이터 주고 받기 데이터 2GB 쿠폰 가족사랑 데이터 링(홀) 요금제 마이샵 사전예약내역 가입 신청 내역 액세서리 주문 내역 최근 본 상품 나의 문의/리뷰 입고알림 ez포인트 ez포인트 안내 ez포인트 조회 ez포인트 사용 혜택/멤버십 멤버십 나의 멤버십 유플투쁠 멤버십 사용처 영화 예매 멤버십 이용내역 장기고객/유쓰 혜택 장기고객 혜택 유쓰 혜택 이벤트 진행 중 이벤트 출석체크 이벤트 지

In [36]:
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-4o"
)

llm_transformer = LLMGraphTransformer(
    llm=llm,
    # allowed_nodes=["Person", "Country", "Organization"],
    # allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "SPOUSE"],
    # node_properties=["born_year"]
)

graph_documents = llm_transformer.convert_to_graph_documents(documents)
graph_documents

[GraphDocument(nodes=[Node(id='Lg Uplus', type='Organization', properties={}), Node(id='황현식', type='Person', properties={}), Node(id='서울특별시 용산구 한강대로 32', type='Location', properties={}), Node(id='220-81-39938', type='Identifier', properties={}), Node(id='2015-서울용산-00481호', type='Identifier', properties={})], relationships=[Relationship(source=Node(id='Lg Uplus', type='Organization', properties={}), target=Node(id='황현식', type='Person', properties={}), type='CEO', properties={}), Relationship(source=Node(id='Lg Uplus', type='Organization', properties={}), target=Node(id='서울특별시 용산구 한강대로 32', type='Location', properties={}), type='HEADQUARTERS', properties={}), Relationship(source=Node(id='Lg Uplus', type='Organization', properties={}), target=Node(id='220-81-39938', type='Identifier', properties={}), type='BUSINESS_REGISTRATION_NUMBER', properties={}), Relationship(source=Node(id='Lg Uplus', type='Organization', properties={}), target=Node(id='2015-서울용산-00481호', type='Identifier', propert

In [37]:
graph.add_graph_documents(graph_documents)